  **CS 6375 Assignment-3, Part_2 : K-Means clustering using Jaccard distance**

  **Name of students: **
  Mounika B(MXB210007)
  Saketh Dasavathini(SXD190016)


In [34]:
#Importing the required libraries

import random as rd
import re
import math
import string
import urllib.request

**Data pre-processing of tweets generated related to bbchealth**

In [45]:
def pre_process(url):
    # The bbchealth.txt file is uploaded in github at the given location below.
    url = 'https://raw.githubusercontent.com/bmounikareddy98/Machine-learning-assignments/main/Assignment3/bbchealth.txt'
    file = urllib.request.urlopen(url)
    bbc_health_tweets = []
    for line in file:
       bbc_health_tweets.append(line)
    
    preprocessed_tweets_list = []

    for i in range(len(bbc_health_tweets)):

        
        bbc_health_tweets[i] = bbc_health_tweets[i].decode('UTF-8') # Converting the byte format of tweet into string, which will automatically removes the \n character that the end
        
        # Remove the tweet id and timestamp
        bbc_health_tweets[i] = bbc_health_tweets[i][50:]

       
        bbc_health_tweets[i] = " ".join(filter(lambda x: x[0] != '@', bbc_health_tweets[i].split()))  # Remove any word that starts with the symbol @

        # Remove any URL using the regular expression
        bbc_health_tweets[i] = re.sub(r"http\S+", "", bbc_health_tweets[i])
        bbc_health_tweets[i] = re.sub(r"www\S+", "", bbc_health_tweets[i])

        # removing the colons from the end of the sentences, after removing url
        bbc_health_tweets[i] = bbc_health_tweets[i].strip()
        tweet_len = len(bbc_health_tweets[i])
        if tweet_len > 0:
            if bbc_health_tweets[i][len(bbc_health_tweets[i]) - 1] == ':':
                bbc_health_tweets[i] = bbc_health_tweets[i][:len(bbc_health_tweets[i]) - 1]

        # Removing any hash-tags symbols
        bbc_health_tweets[i] = bbc_health_tweets[i].replace('#', '')

        # Convert every word to lowercase
        bbc_health_tweets[i] = bbc_health_tweets[i].lower()

        # remove punctuations
        bbc_health_tweets[i] = bbc_health_tweets[i].translate(str.maketrans('', '', string.punctuation))

        # trim extra spaces
        bbc_health_tweets[i] = " ".join(bbc_health_tweets[i].split())

        # convert each tweet from string type to as list<string> using " " as a delimiter
        preprocessed_tweets_list.append(bbc_health_tweets[i].split(' '))

    file.close()

    return preprocessed_tweets_list

**Defining K_means algorithm using Jaccard distance**

In [46]:
def k_means(bbc_health_tweets, k=4, max_iterations=60):

    centroids = []

    
    count = 0
    map = dict() # creating a dictionary to stor key value pairs
    while count < k:
        tweet_index = rd.randint(0, len(bbc_health_tweets) - 1) #assigning random tweets as centroids
        if tweet_index not in map:
            count += 1
            map[tweet_index] = True
            centroids.append(bbc_health_tweets[tweet_index])

    iterations_count = 0
    previous_centroids = []

    # running the iterations until they are not converged or till the max iterations count in not reached
    while (is_k_means_converged(previous_centroids, centroids)) == False and (iterations_count < max_iterations):

        print("running iteration " + str(iterations_count))

        # assignment, assign tweets to the closest centroids
        clusters = assign_the_cluster(bbc_health_tweets, centroids)

        # to check if k-means converges, keep track of previous_centroids
        previous_centroids = centroids

        # update, update centroid based on clusters formed
        centroids = updation_of_centroids(clusters)
        iterations_count = iterations_count + 1

    if (iterations_count == max_iterations):
        print("K Means is not converged as maximum iterations count is reached")
    else:
        print("K Means is converged")

    SSE = compute_SSE(clusters)

    return clusters, SSE

**Checking for the convergence of k-means**

In [47]:
def is_k_means_converged(previous_centroids, new_centroids):

    # If lengths are not equal, we will return false
    if len(previous_centroids) != len(new_centroids):
        return False

    #Going over through each entry of clusters and checking if they are same
    for c in range(len(new_centroids)):
        if " ".join(new_centroids[c]) != " ".join(previous_centroids[c]):
            return False

    return True

**Assigning tweets to clusters according to nearest centroid**

In [48]:
def assign_the_cluster(bbc_health_tweets, centroids):

    clusters = dict()

    # for every health tweet, calculating closest centroid and assigning the tweet to that cluster
    for i in range(len(bbc_health_tweets)):
        min_distance = math.inf
        cluster_index = -1;
        for c in range(len(centroids)):
            dis = getJaccardDistance(centroids[c], bbc_health_tweets[i])
            # look for a closest centroid for a tweet

            if centroids[c] == bbc_health_tweets[i]:
                cluster_index = c
                min_distance = 0
                break

            if dis < min_distance:
                cluster_index = c
                min_distance = dis

        # If we dont have anything in common, we will randomly assign the tweet to cluster
        if min_distance == 1:
            cluster_index = rd.randint(0, len(centroids) - 1)

        # assigning the closest centroid to a tweet
        clusters.setdefault(cluster_index, []).append([bbc_health_tweets[i]])
       
        last_tweet_index = len(clusters.setdefault(cluster_index, [])) - 1
        clusters.setdefault(cluster_index, [])[last_tweet_index].append(min_distance)

    return clusters

In [49]:
def updation_of_centroids(clusters):

    centroids = []

    # iterate through each cluster and check for a health tweet with closest distance and with all other tweets in the same cluster
    # select that tweet as the centroid for the cluster
    for i in range(len(clusters)):
        min_distance_sum = math.inf
        centroid_index = -1

        # to avoid redundant calculations
        min_distance_dp = []

        for t1 in range(len(clusters[i])):
            min_distance_dp.append([])
            distance_sum = 0
            # get distances sum for every of tweet t1 with every tweet t2 in a same cluster
            for t2 in range(len(clusters[i])):
                if t1 != t2:
                    if t2 < t1:
                        distance = min_distance_dp[t2][t1]
                    else:
                        distance = getJaccardDistance(clusters[i][t1][0], clusters[i][t2][0])

                    min_distance_dp[t1].append(distance)
                    distance_sum += distance
                else:
                    min_distance_dp[t1].append(0)

            # The tweet with the minimum distances sum as the centroid for the cluster is selected
            if distance_sum < min_distance_sum:
                min_distance_sum = distance_sum
                centroid_index = t1

        # appending the selected tweet to the centroid list
        centroids.append(clusters[i][centroid_index][0])

    return centroids

**Calculating the Jaccard distance**

In [50]:
def getJaccardDistance(tweet1, tweet2):

    # get the intersection
    intersection = set(tweet1).intersection(tweet2)

    # get the union
    union = set().union(tweet1, tweet2)

    # return the jaccard distance
    return 1 - (len(intersection) / len(union))







**Compute the SSE**

In [51]:
def compute_SSE(clusters):

    sse = 0
    # iterate through each cluster 'c' and compute the sum of square of distances of the tweet from it's centroid
    for c in range(len(clusters)):
        for t in range(len(clusters[c])):
            sse = sse + (clusters[c][t][1] * clusters[c][t][1])

    return sse

**Main function consisting of all the function calls above **

In [52]:
if __name__ == '__main__':

    data_url = 'https://raw.githubusercontent.com/bmounikareddy98/Machine-learning-assignments/main/Assignment3/bbchealth.txt'

    bbc_health_tweets = pre_process(data_url)

    # default number of experiments to be performed using different values of k
    experiments = 5

    # initial value of k for K-means
    k = 3

    # for every experiment 'e', run K-means
    for e in range(experiments):

        print("Running K means for experiment no. " + str((e + 1)) + " for k = " + str(k))

        clusters, sse = k_means(bbc_health_tweets, k)

        # for every cluster 'c', print size of each cluster
        for c in range(len(clusters)):
            print(str(c+1) + ": ", str(len(clusters[c])) + " tweets")
           
        print("--> SSE : " + str(sse))
        print('\n')

        # incrementing k after every experiment
        k = k + 1

Running K means for experiment no. 1 for k = 3
running iteration 0
running iteration 1
running iteration 2
K Means is converged
1:  907 tweets
2:  1569 tweets
3:  1453 tweets
--> SSE : 3405.1838451701815


Running K means for experiment no. 2 for k = 4
running iteration 0
running iteration 1
K Means is converged
1:  1197 tweets
2:  714 tweets
3:  1248 tweets
4:  770 tweets
--> SSE : 3410.1369310634163


Running K means for experiment no. 3 for k = 5
running iteration 0
running iteration 1
running iteration 2
K Means is converged
1:  477 tweets
2:  1531 tweets
3:  624 tweets
4:  682 tweets
5:  615 tweets
--> SSE : 3327.0019087411424


Running K means for experiment no. 4 for k = 6
running iteration 0
running iteration 1
running iteration 2
K Means is converged
1:  828 tweets
2:  280 tweets
3:  644 tweets
4:  1115 tweets
5:  472 tweets
6:  590 tweets
--> SSE : 3327.1853256754716


Running K means for experiment no. 5 for k = 7
running iteration 0
running iteration 1
K Means is converged


In [16]:
import urllib.request
data_url = 'https://raw.githubusercontent.com/bmounikareddy98/Machine-learning-assignments/main/Assignment3/bbchealth.txt'
 #pre_process_tweets(data_url)
#f = open(data_url, "r", encoding="utf8")
file = urllib.request.urlopen(data_url)

tweets=[]
"""for line in file:
  print(line)
  tweets.append(line)"""
for line in file:
  tweets.append(line)
"""for line in file:
	decoded_line = line.decode("utf-8")
  tweets.append(line)
	#print(decoded_line)
  
file.read("r")"""

#tweets=list(file)
#tweets[0]= tweets[0].replace('\n',"")
tweets[0] = tweets[0].decode('UTF-8')
print(type(tweets[0]))
print(tweets[0])
print(type(tweets[0]))
#str(b, encoding)

#print(a)
#for line in file:
  #print(line)

    
#print(file.read)


<class 'str'>
585978391360221184|Thu Apr 09 01:31:50 +0000 2015|Breast cancer risk test devised http://bbc.in/1CimpJF

<class 'str'>


In [69]:
#tweets[0] = tweets[0].strip('\n')
print(tweets[0])

b'585978391360221184|Thu Apr 09 01:31:50 +0000 2015|Breast cancer risk test devised http://bbc.in/1CimpJF\r\n'
